In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
import xgboost as xgb
import re
from sklearn.model_selection import GridSearchCV
import warnings
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_table("cleaned_data.csv", sep=",", encoding="utf-8")
print(data.columns)

Index(['PNO', 'total', 'BED_chair', 'CDATE', 'PLACE', 'ITEM', 'ORDERDR',
       'MD_NO', 'CASENO', 'AMOUNT', 'IO', 'DEPT', 'Btime', 'Etime', 'SEX',
       'AGE', 'POS', 'MODEL_NAME', 'DATE', 'time_label', 'YM'],
      dtype='object')


In [3]:
# build_xgb(data):
# get_mean_rank

select = ["ITEM", "ORDERDR", 'BED_chair',
          "DEPT", "POS", "SEX", 'PLACE',
          "MODEL_NAME", "IO"]

for col in select:
    value = data['total'].groupby(data[col]).mean().rank() - 1
    exec("%s = value.to_dict()" % col)  # 動態變數

# 新增欄位 內容為排名 -> mean_rank
for col in select:
    data[col + "_n"] = data[col].apply(lambda x: eval("%s[x]" % str(col)))
    
# 控制建模放入的變數

select = ["ITEM_n", "ORDERDR_n","DEPT_n", 
          "total","SEX_n", "POS_n","AGE",
          "PLACE_n", "IO_n", 'PNO']

data = data[select]
check = data.columns

In [ ]:
data

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['total', 'PNO'], axis=1), data[['total','PNO']], test_size=0.3)
#
y_train = y_train.drop(['PNO'], axis=1)
#
X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
#
MR_params = {'n_estimators': [35,36,37,38,39,40]}
MR_params = {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_child_weight': [1, 2, 3, 4, 5, 6]}
MR_params = {'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]}
MR_params = {'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]}
MR_params = {'reg_alpha': [0.05, 0.1, 1, 2, 3], 'reg_lambda': [0.05, 0.1, 1, 2, 3]}
MR_params = {'learning_rate': [0.01, 0.05, 0.1, 0.2]}
#調動上面修改下面 一行一行調 每調動一次請把最佳結果填入下面參數
# mr_params_list = [{'n_estimators': [35,36,37,38,39,40]},
#                  {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_child_weight': [1, 2, 3, 4, 5, 6]},
#                  {'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]},
#                  {'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]},
#                  {'reg_alpha': [0.05, 0.1, 1, 2, 3], 'reg_lambda': [0.05, 0.1, 1, 2, 3]},
#                  {'learning_rate': [0.01, 0.05, 0.1, 0.2]}]
# mr_params_dict = {
#     'n_estimators': [35,36,37,38,39,40],
#     'max_depth': [3, 4, 5, 6, 7, 8],
#     'min_child_weight': [ 2, 3, 4, 5, 6],
#     'gamma': [0.1, 0.2, 0.3, 0.4, 0.5],
#     'subsample': [0.6, 0.7, 0.8, 0.9],
#     'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
#     'reg_alpha': [0.05, 0.1, 1, 2, 3],
#     'reg_lambda': [0.05, 0.1, 1, 2, 3],
#     'learning_rate': [0.01, 0.05, 0.1, 0.2]
# }
other_params = {'learning_rate': 0.1, 'n_estimators': 39, 'max_depth': 4, 'min_child_weight': 5, 
                'seed': 0,'subsample': 0.6, 'colsample_bytree': 0.8, 'gamma': 0.1, 
                'reg_alpha': 3, 'reg_lambda': 0.1}
gbm = xgb.XGBRegressor(**other_params)

optimized_GBM = GridSearchCV(estimator=gbm, param_grid=MR_params, 
                             scoring='r2', cv=5, verbose=1, n_jobs=-1)
optimized_GBM.fit(X_train, y_train)
evalute_result = optimized_GBM.cv_results_
print('引數的最佳取值：{0}'.format(optimized_GBM.best_params_))
# print('每輪迭代執行結果:{0}'.format(evalute_result))
# print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

Fitting 5 folds for each of 1440000 candidates, totalling 7200000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 1476 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done 2376 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done 3476 tasks      | elapsed:   49.6s
[Parallel(n_jobs=-1)]: Done 4776 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 6276 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 7976 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 9876 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 11976 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 14276 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 16776 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 19476 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 22376 tasks  

[Parallel(n_jobs=-1)]: Done 1587476 tasks      | elapsed: 434.3min
[Parallel(n_jobs=-1)]: Done 1612776 tasks      | elapsed: 442.2min
[Parallel(n_jobs=-1)]: Done 1638276 tasks      | elapsed: 448.7min
[Parallel(n_jobs=-1)]: Done 1663976 tasks      | elapsed: 455.3min
[Parallel(n_jobs=-1)]: Done 1689876 tasks      | elapsed: 463.1min
[Parallel(n_jobs=-1)]: Done 1715976 tasks      | elapsed: 471.1min
[Parallel(n_jobs=-1)]: Done 1742276 tasks      | elapsed: 477.3min
[Parallel(n_jobs=-1)]: Done 1768776 tasks      | elapsed: 484.7min
[Parallel(n_jobs=-1)]: Done 1795476 tasks      | elapsed: 493.5min
[Parallel(n_jobs=-1)]: Done 1822376 tasks      | elapsed: 500.3min
[Parallel(n_jobs=-1)]: Done 1849476 tasks      | elapsed: 507.7min
[Parallel(n_jobs=-1)]: Done 1876776 tasks      | elapsed: 516.0min
[Parallel(n_jobs=-1)]: Done 1904276 tasks      | elapsed: 523.6min
[Parallel(n_jobs=-1)]: Done 1931976 tasks      | elapsed: 531.0min
[Parallel(n_jobs=-1)]: Done 1959876 tasks      | elapsed: 539.

[Parallel(n_jobs=-1)]: Done 6150276 tasks      | elapsed: 1832.4min
[Parallel(n_jobs=-1)]: Done 6199976 tasks      | elapsed: 1848.9min
[Parallel(n_jobs=-1)]: Done 6249876 tasks      | elapsed: 1863.6min
[Parallel(n_jobs=-1)]: Done 6299976 tasks      | elapsed: 1882.3min
[Parallel(n_jobs=-1)]: Done 6350276 tasks      | elapsed: 1896.7min
[Parallel(n_jobs=-1)]: Done 6400776 tasks      | elapsed: 1914.8min
[Parallel(n_jobs=-1)]: Done 6451476 tasks      | elapsed: 1930.4min
[Parallel(n_jobs=-1)]: Done 6502376 tasks      | elapsed: 1947.6min
[Parallel(n_jobs=-1)]: Done 6553476 tasks      | elapsed: 1964.0min
[Parallel(n_jobs=-1)]: Done 6604776 tasks      | elapsed: 1979.4min
[Parallel(n_jobs=-1)]: Done 6656276 tasks      | elapsed: 1998.3min
[Parallel(n_jobs=-1)]: Done 6707976 tasks      | elapsed: 2013.4min
[Parallel(n_jobs=-1)]: Done 6759876 tasks      | elapsed: 2032.0min
[Parallel(n_jobs=-1)]: Done 6811976 tasks      | elapsed: 2048.1min
[Parallel(n_jobs=-1)]: Done 6864276 tasks      |

引數的最佳取值：{'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 6, 'n_estimators': 37, 'reg_alpha': 3, 'reg_lambda': 3, 'subsample': 0.6}


In [6]:
print(optimized_GBM.best_params_)

{'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 6, 'n_estimators': 37, 'reg_alpha': 3, 'reg_lambda': 3, 'subsample': 0.6}


In [ ]:
# print('每輪迭代執行結果:{0}'.format(evalute_result))
# print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

In [9]:
gbm = xgb.XGBRegressor(learning_rate=0.1, n_estimators=38, max_depth=4,
                       min_child_weight=5, seed=0, subsample=0.6, colsample_bytree=0.6, 
                       gamma=0.1, reg_alpha=2, reg_lambda=2)
gbm.fit(X_train, y_train)
predictions = gbm.predict(X_test)

In [10]:
def MAPE(true, pred):
    
    diff = np.abs(np.array(true) - np.array(pred))
    global y_test
    y_test['pred'] = pred.astype(int)
    y_test['diff']= diff.astype(int)
    print(y_test)
    return np.mean(diff / true)
print('xgb模型的MAPE :', MAPE(y_test['total'].astype(int), predictions))

       total       PNO  pred  diff
148   2100.0  45861469  2281   181
2899  3000.0  41996068  2525   474
1146  2700.0  24473521  2424   275
2365  2400.0  46962762  2283   116
2515  1800.0  52743046  2355   555
283   1800.0  50876690  4085  2285
996   1500.0  31846925  2052   552
1799  2400.0   7702618  2039   360
2630  2400.0  22370503  2510   110
1906  3300.0  36194581  3062   237
4776  2400.0  18713407  1991   408
134   3300.0  51166587  2428   871
5308  3000.0  18372498  2420   579
3221  3000.0  31601489  2485   514
4289  3000.0  48768446  2045   954
1086  3300.0  10383915  2523   776
2905  2280.0  25052537  2493   213
4241  1800.0  41394548  2050   250
1569  1800.0  52347273  2388   588
2520  1500.0   6327364  1983   483
425   3000.0  25198322  1985  1014
4163  3000.0  25163872  2523   476
2748  2400.0  52848144  2659   259
4205  2400.0  43502898  2525   125
4666  3000.0   8473163  2493   506
4475  1800.0  45843041  2281   481
3926  1800.0  39873062  2179   379
2283  2400.0   14648

In [ ]:
def MAPE(true, pred):
    
    diff = np.abs(np.array(true) - np.array(pred))
    global y_test
    y_test['pred'] = pred.astype(int)
    y_test['diff']= diff.astype(int)
    
    print(y_test)
    
    y_test.to_csv('Result.csv')
#     print("np.array(true): ", np.array(true).astype(int))

    np.savetxt("true3.csv", true, delimiter=",")
#     print(type(true.astype(int)))
    
#     print("np.array(pred): ", np.array(pred))
    np.savetxt("pred3.csv", np.array(pred), delimiter=",")
    
#     print("diff: ", diff)
    np.savetxt("diff3.csv", np.array(diff), delimiter=",")
    
#     print("y_test: ", true)
    return np.mean(diff / true)

# 模型估計的mape
print('xgb模型的MAPE :', MAPE(y_test['total'].astype(int), predictions))


In [ ]:
y_test.sort_values(by=['diff'])

In [ ]:
y_test[y_test['PNO']=='498745']#實際時間遠大於預測時間 AGE:52

In [ ]:
y_test[y_test['PNO']=='52327940']#同樣條件下的預測表現

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

df = y_test.copy()
# print(df)
ax = sns.barplot(x="PNO", y="pred",color='pink', data=df[0:50])
ax = sns.barplot(x="PNO", y="total",facecolor=(1, 1, 1, 0),edgecolor=".2", data=df[0:50])
ax.figure.savefig("output.png")

In [ ]:
pickle.dump(gbm, open("pima_20190915_xgb.pickle", "wb"))

In [ ]:

# 以下為原始碼，以上為破解版
# 建模

# X_train, X_test, y_train, y_test = \
#     train_test_split(data.drop(['total'], axis=1), data['total'], test_size=0.3)
# #
# X_train = X_train.as_matrix()
# X_test = X_test.as_matrix()
# #

# gbm = xgb.XGBRegressor(objective="reg:linear", n_estimators=50, learning_rate=0.05).fit(X_train, y_train)
# predictions = gbm.predict(X_test)

# pickle.dump(gbm, open("pima.pickle.dat", "wb"))

# def MAPE(true, pred):
#     diff = np.abs(np.array(true) - np.array(pred))
#     return np.mean(diff / true)

# # 模型估計的mape
# print('xgb模型的MAPE :', MAPE(y_test, predictions))

# X_test = data.drop(['total'], axis=1)
# X_test = X_test.as_matrix()

# submission['total'] = gbm.predict(X_test)
# def for_simulation():
#     submission['prediction'] = gbm.predict(X_test)
#     submission.to_csv('mr_with_predictions.csv')
# for_simulation()
# hospital_predict = [40, 48, 53]
# def MAPE(true, h):
#     diff = np.abs(np.array(true) - h*60)
#     return np.mean(diff / true)
# # 如果假設醫護人員每個人都估40分鐘，其mape:
# for h in hospital_predict:
#     print('醫院估計病人:', h, '分鐘', MAPE(y_test, h))




# return submission
